In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client["class"]
collection = db.yeogi_dangjin

service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("window-size=1920x1080")
options.add_argument("--disable-gpu")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36 Edg/138.0.0.0")
options.add_argument("lang=ko_KR")

driver = webdriver.Chrome(service=service, options=options)

base_url = "https://www.yeogi.com/domestic-accommodations?keyword=%EB%8B%B9%EC%A7%84&checkIn=2025-10-08&checkOut=2025-10-09&personal=2&freeForm=true"

for page_num in range(1,4) :
    if page_num == 1 :
        url = base_url
    else : 
        url = base_url + f"&page={page_num}"

    driver.get(url)
    time.sleep(3)

    names = driver.find_elements(By.CSS_SELECTOR, "h3.gc-thumbnail-type-seller-card-title")
    urls = driver.find_elements(By.CSS_SELECTOR, "a.gc-thumbnail-type-seller-card")
    location = driver.find_elements(By.CSS_SELECTOR, "span.css-ki0lqh")
    price_min = driver.find_elements(By.CSS_SELECTOR, "span.css-5r5920")
    ratings =  driver.find_elements(By.CSS_SELECTOR, "span.css-9ml4lz")
    review_counts = driver.find_elements(By.CSS_SELECTOR, "span.css-oj6onp")

    for i in range(len(names)) :
        try :
            data = {
                "name" : names[i].text,
                "url" : urls[i].get_attribute("href"),
                "location" : location[i].text,
                "price_min" : price_min[i].text,
                "rating" : ratings[i].text,
                "review_count" : review_counts[i].text
            }
            print(data)
            collection.insert_one(data)
        except Exception as e:
            print(f"{i+1}번째 숙소 정보가 누락됨, 오류 : {e}")

client.close()
driver.quit()


{'name': '당진 호텔리움', 'url': 'https://www.yeogi.com/domestic-accommodations/5692?checkIn=2025-10-08&checkOut=2025-10-09&personal=2', 'location': '당진시 읍내동', 'price_min': '20,000', 'rating': '9.5', 'review_count': '297명 평가'}
{'name': '당진 저스트슬립 호텔-서해대교점', 'url': 'https://www.yeogi.com/domestic-accommodations/4574?checkIn=2025-10-08&checkOut=2025-10-09&personal=2', 'location': '당진시 송악읍', 'price_min': '70,000', 'rating': '9.8', 'review_count': '471명 평가'}
{'name': '당진 송스테이', 'url': 'https://www.yeogi.com/domestic-accommodations/87720?checkIn=2025-10-08&checkOut=2025-10-09&personal=2', 'location': '당진시 채운동', 'price_min': '25,000', 'rating': '10', 'review_count': '6명 평가'}
{'name': '당진 두바이 호텔', 'url': 'https://www.yeogi.com/domestic-accommodations/13507?checkIn=2025-10-08&checkOut=2025-10-09&personal=2', 'location': '당진시 송악읍', 'price_min': '55,000', 'rating': '9.6', 'review_count': '463명 평가'}
{'name': '당진 저스트슬립호텔 당진터미널점', 'url': 'https://www.yeogi.com/domestic-accommodations/72056?checkIn=2025-10

In [8]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client["class"]
collection = db.yeogi_dangjin

top_10 = collection.aggregate([
    {"$sort" : {"rating" : -1, "review_count" : -1}},
    {"$limit" : 10}
])

for top in top_10 :
    print(top)

client.close()

{'_id': ObjectId('68c2979b436a597a0926e615'), 'name': '당진 저스트슬립 호텔-서해대교점', 'url': 'https://www.yeogi.com/domestic-accommodations/4574?checkIn=2025-10-08&checkOut=2025-10-09&personal=2', 'location': '당진시 송악읍', 'price_min': '70,000', 'rating': '9.8', 'review_count': '471명 평가'}
{'_id': ObjectId('68c2979b436a597a0926e61b'), 'name': '당진 소난지도 해변펜션', 'url': 'https://www.yeogi.com/domestic-accommodations/87294?checkIn=2025-10-08&checkOut=2025-10-09&personal=2', 'location': '소난지도 도보 1분', 'price_min': '60,000', 'rating': '9.8', 'review_count': '12명 평가'}
{'_id': ObjectId('68c2979b436a597a0926e619'), 'name': '당진 덴스테이-한진포구점', 'url': 'https://www.yeogi.com/domestic-accommodations/80313?checkIn=2025-10-08&checkOut=2025-10-09&personal=2', 'location': '당진시 송악읍', 'price_min': '60,000', 'rating': '9.8', 'review_count': '116명 평가'}
{'_id': ObjectId('68c2979b436a597a0926e61a'), 'name': '당진 큐비스트호텔', 'url': 'https://www.yeogi.com/domestic-accommodations/2899?checkIn=2025-10-08&checkOut=2025-10-09&personal=2',